### Import Libraries

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from nsepython import *
from scipy.stats import iqr # For Finding Volume Outliers
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Read the CSV file

In [2]:
## Stock Symbols contains list of 1250 NSE stock symbols (Need to update it every quarter / year)

stock_data = pd.read_csv('stock_data.csv')
stock_data.set_index('SYMBOL', inplace=True)
len(stock_data)

1435

In [3]:
momentum_stocks = pd.DataFrame({'SYMBOL':[], '20EMA':[], '40EMA':[], '100EMA':[], '200EMA':[],
                                'MARKET_CAP':[], 'INDUSTRY': [], 'SECTOR': [], 'MACRO':[], 'FNO':[]})

for symbol in stock_data.index.values:
    try:
        stock = yf.download(symbol + ".NS", start=(pd.to_datetime('today') - pd.DateOffset(days=365)), end=pd.to_datetime('today'), progress=False)
        stock_length = len(stock['Close'])
        
        if stock_length > 100:
            close_price = stock['Close'].dropna()
            
            ema_20 = close_price.ewm(span=20, adjust=False).mean()
            ema_40 = close_price.ewm(span=40, adjust=False).mean()
            ema_100 = close_price.ewm(span=100, adjust=False).mean()
            ema_200 = close_price.ewm(span=200, adjust=False).mean()
            
            ## Logic 
            ## 1. Latest Close Price is greater than 20 Day exponential moving average
            ## 2. 20 Day ema greater than 40 Day ema
            ## 3. 40 Day ema greater than 100 Day ema
            ## 4. 100 Day ema greater than 200 Day ema
            
            if (close_price.iloc[-1] > ema_20.iloc[-1] and
                ema_20.iloc[-1] > ema_40.iloc[-1] and
                ema_40.iloc[-1] > ema_100.iloc[-1] and
                ema_100.iloc[-1] > ema_200.iloc[-1]):
                
                new_stock = pd.DataFrame({'SYMBOL': [symbol], 
                                          '20EMA':[ema_20.iloc[-1]], 
                                          '40EMA':[ema_40.iloc[-1]], 
                                          '100EMA':[ema_100.iloc[-1]], 
                                          '200EMA':[ema_200.iloc[-1]],
                                          'MARKET_CAP':[stock_data.loc[symbol]['MARKET_CAP']], 
                                          'INDUSTRY': [stock_data.loc[symbol]['INDUSTRY']], 
                                          'SECTOR': [stock_data.loc[symbol]['SECTOR']], 
                                          'MACRO':[stock_data.loc[symbol]['MACRO']], 
                                          'FNO':[stock_data.loc[symbol]['FNO']]})
                momentum_stocks = pd.concat([momentum_stocks, new_stock], ignore_index=True)
                
    except Exception as e:
        print(f"An error occurred: {symbol}, {str(e)}")

print(f"Execution completed.")

Execution completed.


BREAK

In [16]:
momentum_high_volume_stocks = pd.DataFrame({'SYMBOL': [], 'FIRST_HIGH_VOLUME': [], 
                                            'LAST_HIGH_VOLUME': [], 'HIGH_VOL_COUNT': [],
                                            '20EMA':[], '40EMA':[], '100EMA':[], '200EMA':[],
                                            'MARKET_CAP':[], 'INDUSTRY': [], 'SECTOR': [], 'MACRO':[], 'FNO':[]})

for index, row in momentum_stocks.iterrows():
    stock = yf.download(row['SYMBOL'] + ".NS", start=(pd.to_datetime('today') - pd.DateOffset(days=90)), end=pd.to_datetime('today'), progress=False)
    iqr_stock = iqr(stock['Volume'])
    volume_cutoff = np.quantile(stock['Volume'], 0.75) + 3 * iqr_stock ## To find Outliers
    high_volume_stock = stock[stock['Volume'] > volume_cutoff]
    
    # convert this data into a row
    if(high_volume_stock.size > 0):
        new_stock = pd.DataFrame({'SYMBOL': [row['SYMBOL']], 
                                  'FIRST_HIGH_VOLUME': [high_volume_stock.index[0]], 
                                  'LAST_HIGH_VOLUME': [high_volume_stock.index[high_volume_stock.shape[0]-1]], 
                                  'HIGH_VOL_COUNT': [high_volume_stock.shape[0]],
                                  '20EMA':[row['20EMA']],
                                  '40EMA':[row['40EMA']],
                                  '100EMA':[row['100EMA']],
                                  '200EMA':[row['200EMA']],
                                  'MARKET_CAP':[row['MARKET_CAP']], 
                                  'INDUSTRY': [row['INDUSTRY']], 
                                  'SECTOR': [row['SECTOR']], 
                                  'MACRO':[row['MACRO']], 
                                  'FNO':[row['FNO']]})
        momentum_high_volume_stocks = pd.concat([momentum_high_volume_stocks, new_stock], ignore_index=True)
print(f"Execution completed with {momentum_high_volume_stocks.shape[0]} stocks")

Execution completed with 573 stocks


BREAK

In [ ]:
momentum_high_volume_stocks_reasonable_recent = momentum_high_volume_stocks[momentum_high_volume_stocks['LAST_HIGH_VOLUME'] > (pd.to_datetime('today') - pd.DateOffset(days=30))]
momentum_high_volume_stocks_reasonable_recent = momentum_high_volume_stocks_reasonable_recent[
    (momentum_high_volume_stocks_reasonable_recent['20EMA'] < 1.15 * momentum_high_volume_stocks_reasonable_recent['40EMA']) & 
    (momentum_high_volume_stocks_reasonable_recent['40EMA'] < 1.2 * momentum_high_volume_stocks_reasonable_recent['100EMA'])]
momentum_high_volume_stocks_reasonable_recent = momentum_high_volume_stocks_reasonable_recent[
    (momentum_high_volume_stocks_reasonable_recent['MARKET_CAP'] > 1000) &
    (momentum_high_volume_stocks_reasonable_recent['MARKET_CAP'] < 100000)
]

In [ ]:
stock_data['INDUSTRY'].unique()

In [15]:
# Plot Recent Momentum High Volume Stocks

## Actual Code Begins Here

# potential_stocks = [symbol for symbol in stock_data.index.values if symbol not in momentum_stocks['SYMBOL'].values]
# pot_stock_data = stock_data[stock_data.index.isin(potential_stocks)]
# pot_stock_data = pot_stock_data[pot_stock_data['MARKET_CAP'] > 1000]
# pot_stock_data.groupby('SECTOR').count().sort_values('MARKET_CAP', ascending=False)


# plot_symbols = recent_momentum_high_volume_stocks.iloc[150:200]['SYMBOL'].values
# plot_symbols = stock_data[stock_data['INDUSTRY'].isin(['Metals & Minerals Trading','Ferrous Metals','Non - Ferrous Metals','Minerals & Mining'])].index.values

plot_symbols = momentum_high_volume_stocks.sort_values('HIGH_VOL_COUNT', ascending = False).iloc[190:194]['SYMBOL'].values

try:
    figure_html = open('momentum_volume.html', 'w')

    for symbol in ['IFCI']:
        
        try:
            if(symbol not in stock_data.index):
                print(f"{symbol} is not part of our stock data!")
                continue
            data = nse_past_results(symbol)

            df = pd.DataFrame(data.get('resCmpData'))
            isBank = data.get('bankNonBnking') == 'B'
    
            if(not isBank):
                if len(df.columns) == 0:
                    print(f"{symbol} no past results found for this symbol")
                    continue

                new_columns = {
                    're_to_dt': 'DATE',
                    're_create_dt': 'RELEASE_DATE',
                    're_net_sale': 'INCOME',
                    're_oth_tot_exp': 'COST',
                    're_oth_inc_new': 'OTHER_INCOME',
                    're_pro_loss_bef_tax': 'PBT',
                    're_net_profit': 'PROFIT',
                    're_basic_eps_for_cont_dic_opr': 'EPS'
                }

                df.rename(columns=new_columns, inplace=True)

                df = df[['DATE', 'RELEASE_DATE', 'INCOME', 'COST', 'OTHER_INCOME', 'PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)

                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])

                for col in df.columns:
                    if col not in ['EPS', 'RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')

                df['OP_INCOME'] = df['INCOME'] - df['COST']
                df['OPM'] = (100 * (df['INCOME'] - df['COST']) / df['INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"

                df = df[['INCOME', 'OP_INCOME', 'OPM', 'OTHER_INCOME', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]

                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
            else:
                new_columns = {'re_to_dt': 'DATE', 
                   're_create_dt': 'RELEASE_DATE', 
                   're_int_earned':'NET_INTEREST_INCOME',
                   're_tot_exp_exc_pro_cont':'OPERATING_EXPENSE', 
                   're_oth_inc':'OTHER_INCOME',
                   're_oth_pro_cont':'PROVISION',
                   're_pro_loss_bef_tax':'PBT', 
                   're_con_pro_loss':'PROFIT', 
                   're_basic_eps':'EPS'}
    
                df.rename(columns=new_columns, inplace=True)
    
                df = df[['DATE', 'RELEASE_DATE', 'NET_INTEREST_INCOME', 'OPERATING_EXPENSE', 'PROVISION', 
                         'OTHER_INCOME','PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)
    
                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])
    
                for col in df.columns:
                    if col not in ['EPS','RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')
            
                df['OP_INCOME'] = df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']
                df['OPM'] = (100 * (df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']) / df['NET_INTEREST_INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"
    
                df = df[['NET_INTEREST_INCOME', 'OP_INCOME', 'OPM','OTHER_INCOME', 'PROVISION', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]
    
                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
        
    
            ### Ploting 
    
            data = yf.download(symbol + ".NS", start='2023-01-01', end=pd.to_datetime('today')+pd.DateOffset(1), progress=False)
    
            data['20DMA'] = data['Close'].rolling(window=20).mean()
            data['50DMA']= data['Close'].rolling(window=50).mean()
            data['100DMA']= data['Close'].rolling(window=100).mean()
    
            data['diff'] = data['Close'] - data['Open']
            data.loc[data['diff'] >= 0, 'color'] = 'green'
            data.loc[data['diff'] < 0, 'color'] = 'red'
    
            plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=270))]

            figure = make_subplots(specs=[[{"secondary_y": True}]])
    
            # Subplot 1 - Price Candlestick
    
            figure.add_trace(go.Candlestick(x = plot_data.index,
                              open = plot_data['Open'],
                              high = plot_data['High'],
                              low = plot_data['Low'],
                              close = plot_data['Close'],
                              name='Price'))
            figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
            figure.update_xaxes(rangebreaks = [ dict(bounds=['sat','mon']) ] ) # hide weekends 
            figure.update_layout(title={'text':symbol, 'x':0.5})
            figure.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
    
            # Subplot 2 - Volume and DMAs
    
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['20DMA'], marker_color='blue',name='20 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['50DMA'], marker_color='orange',name='50 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['100DMA'], marker_color='green',name='100 Day MA'))
    
            figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume', marker={'color':plot_data['color']}),secondary_y=True)
            figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
            figure.update_yaxes(visible=False, secondary_y=True)
    
            # Adding Text Below
            stock_row = stock_data.loc[symbol]
            figure.add_annotation(dict(font=dict(color='black', size=16.5),
                x=0.5,  # Center aligned horizontally
                y=-0.12, showarrow=False,
                text="SECTOR - " + stock_row['SECTOR']+"  MARKET CAP - "+str(stock_row['MARKET_CAP'].astype(int)) + "Cr",
                textangle=0,
                xanchor='center',  # Center aligned horizontally
                yanchor='bottom',  # Aligned to the bottom
                xref="paper", yref="paper"))
    
            # Adding Earnings Date in the figure
            if('31-DEC-2023' in df.columns):
                figure.add_annotation(go.layout.Annotation(text="Q3",
                    x=pd.to_datetime(df.loc['RELEASE_DATE']['31-DEC-2023']),
                    y=plot_data['Close'].min()*0.9, 
                    showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                    ax=0, ay=-40))
    
            figure.add_annotation(go.layout.Annotation(text="Q2",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-SEP-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            figure.add_annotation(go.layout.Annotation(text="Q1",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-JUN-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            html_table = df.style.set_table_styles([
            {'selector': 'thead th', 'props': [('background-color', 'lightblue'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
            {'selector': 'tbody td', 'props': [('text-align', 'center')]}]).render()
    
            ### Writing the results dataframe and plot to the html page
            figure_html.write(html_table)
            figure_html.write(figure.to_html(full_html=False))
        
        except Exception as symbol_exception:
            print(f"Error processing symbol {symbol}: {symbol_exception}")
            continue
except Exception as main_exception:
    print(f"Main error: {main_exception}")

finally:
    # Close the HTML file
    figure_html.close()

print('Execution Completed!!!!')

Execution Completed!!!!


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_21229/710171451.py:180: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



In [ ]:
momentum_high_volume_stocks.sort_values('HIGH_VOL_COUNT', ascending=False).iloc[0:50]

In [17]:
momentum_high_volume_stocks[momentum_high_volume_stocks['SYMBOL'] == 'IFCI']

,SYMBOL,FIRST_HIGH_VOLUME,LAST_HIGH_VOLUME,HIGH_VOL_COUNT,20EMA,40EMA,100EMA,200EMA,MARKET_CAP,INDUSTRY,SECTOR,MACRO,FNO
223,IFCI,2023-12-04,2024-01-24,3.0,32.846589,30.254098,25.587733,21.265745,7232.0,Finance,Financial Services,Financial Services,0.0


In [5]:
momentum_stocks

,SYMBOL,20EMA,40EMA,100EMA,200EMA,MARKET_CAP,INDUSTRY,SECTOR,MACRO,FNO
0,RELIANCE,2662.227249,2591.721365,2506.015862,2471.400238,1750283.0,Petroleum Products,Oil Gas & Consumable Fuels,Energy,1.0
1,TCS,3819.711083,3746.036718,3614.777177,3520.800294,1374873.0,IT - Software,Information Technology,Information Technology,1.0
2,ICICIBANK,997.951234,990.200842,972.626838,952.142651,700333.0,Banks,Financial Services,Financial Services,1.0
3,INFY,1596.304450,1553.109597,1491.836186,1463.396439,640269.0,IT - Software,Information Technology,Information Technology,1.0
4,BHARTIARTL,1080.550533,1043.285917,981.958615,924.599137,579043.0,Telecom - Services,Telecommunication,Telecommunication,1.0
...,...,...,...,...,...,...,...,...,...,...
627,TRIGYN,147.262100,140.595533,131.226907,122.415519,411.0,IT - Software,Information Technology,Information Technology,0.0
628,IL&FSENGG,34.198474,31.377989,26.033890,21.608725,408.0,Construction,Construction,Industrials,0.0
629,IITL,217.952168,201.200296,169.946091,144.859624,404.0,Finance,Financial Services,Financial Services,0.0
630,TPLPLASTEH,57.893354,55.684530,51.331082,46.659275,404.0,Industrial Products,Capital Goods,Industrials,0.0
